In [1]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import PolynomialFeatures
from sklearn import decomposition
import os
import scipy.stats as sc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


In [2]:
df_co = pd.read_csv('Transformed Data/Transformed_Co', index_col = 0)
df_pt = pd.read_csv('Transformed Data/Transformed_Pt' , index_col = 0)

In [3]:
df_co_len = df_co.shape[0]
df_pt_len = df_pt.shape[0]

In [4]:
df_co_pca = pd.DataFrame(df_co)
df_pt_pca = pd.DataFrame(df_pt)

In [5]:
y1 = pd.Series([0]*df_co_len)
y1.shape

(92,)

In [6]:
y2 = pd.Series([1]*df_pt_len, index = range(df_co_len-1,(df_co_len + df_pt_len)-1))
y2.shape

(214,)

In [7]:
y = pd.concat([y1,y2]) 
y.shape

(306,)

In [8]:
X = pd.concat([df_co_pca, df_pt_pca])

In [9]:
max_C = 0
max_acc = 0
curr_max_acc = 0
curr_max_c =0
for i in range (0,1000):
    
    X_train, X_test, y_train1, y_test1 = train_test_split(X, y, random_state=i)
    y_train = pd.DataFrame(y_train1)
    y_test = pd.DataFrame(y_test1)
    
    scaler = MinMaxScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    
    

    for this_C in ([0.01, 0.1, 1, 5, 10, 50, 100]):

        clf = SVC(kernel = 'linear', C = this_C).fit(X_train, np.ravel(y_train))
        
        if(max_acc < clf.score(X_test, np.ravel(y_test))):
            max_acc = clf.score(X_test, np.ravel(y_test))
            max_C=this_C
    if(curr_max_acc < max_acc):
        curr_max_acc = max_acc
        curr_max_c = max_C
        curr_rs = i
    print("{} done".format(i))
print(curr_max_acc,curr_max_c,curr_rs) 
    
    
    
"""print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)"""


0 done
1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done
28 done
29 done
30 done
31 done
32 done
33 done
34 done
35 done
36 done
37 done
38 done
39 done
40 done
41 done
42 done
43 done
44 done
45 done
46 done
47 done
48 done
49 done
50 done
51 done
52 done
53 done
54 done
55 done
56 done
57 done
58 done
59 done
60 done
61 done
62 done
63 done
64 done
65 done
66 done
67 done
68 done
69 done
70 done
71 done
72 done
73 done
74 done
75 done
76 done
77 done
78 done
79 done
80 done
81 done
82 done
83 done
84 done
85 done
86 done
87 done
88 done
89 done
90 done
91 done
92 done
93 done
94 done
95 done
96 done
97 done
98 done
99 done
100 done
101 done
102 done
103 done
104 done
105 done
106 done
107 done
108 done
109 done
110 done
111 done
112 done
113 done
114 done
115 done
116 done
117 done
118 done
119 done
120 done
121 done
122 done
123

924 done
925 done
926 done
927 done
928 done
929 done
930 done
931 done
932 done
933 done
934 done
935 done
936 done
937 done
938 done
939 done
940 done
941 done
942 done
943 done
944 done
945 done
946 done
947 done
948 done
949 done
950 done
951 done
952 done
953 done
954 done
955 done
956 done
957 done
958 done
959 done
960 done
961 done
962 done
963 done
964 done
965 done
966 done
967 done
968 done
969 done
970 done
971 done
972 done
973 done
974 done
975 done
976 done
977 done
978 done
979 done
980 done
981 done
982 done
983 done
984 done
985 done
986 done
987 done
988 done
989 done
990 done
991 done
992 done
993 done
994 done
995 done
996 done
997 done
998 done
999 done
0.948051948052 1 37


'print(X_train.shape)\nprint(X_test.shape)\nprint(y_train.shape)\nprint(y_test.shape)'

In [ ]:

max_score = 0
best_param = []
for i in range(0,1000):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    clf = SVC(kernel = 'linear', C = 1).fit(X_train, np.ravel(y_train))
    from sklearn.model_selection import GridSearchCV

    grid_values = {'C': [1, 2, 5, 10, 15, 20, 25, 40, 50, 70, 100, 500, 1000]}

    grid_lr = GridSearchCV(clf, param_grid = grid_values, scoring = 'accuracy')
    grid_lr.fit(X_train, y_train)
    if max_score < grid_lr.best_score_:
        max_score = grid_lr.best_score_
        best_param = grid_lr.best_params_

In [ ]:
print( max_score)
print( best_param)